In [2]:
# %pip install regex requests torch numpy transformers datasets evaluate rouge_score

In [1]:
!nvidia-smi

Thu Mar  6 12:44:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 4000 Ada Gene...    Off |   00000000:01:00.0 Off |                  Off |
| 30%   56C    P2             53W /  130W |    1919MiB /  20475MiB |     59%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from mingpt.bpe import BPETokenizer
from mingpt.model import GPT

import torch
from torch.utils.data import Dataset
from evaluate import load
from datasets import Dataset
import glob
from datasets import load_dataset
import pandas as pd

# 1. Dataset

In [4]:
class QADataset(Dataset):
    def __init__(self, data, tokenizer, block_size=128):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.data1 = [
            self.format_example(data_point["text"]) for data_point in data
        ]

    def format_example(self, text):
        text = f"{text}"
        tokens = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=self.block_size)
        tokens = tokens['input_ids'].squeeze(0)[:self.block_size]        
        
        return tokens

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, idx):
        """
        Returns:
            x (torch.Tensor): Input tokens (question + answer prompt).
            y (torch.Tensor): Target tokens (shifted output).
        """
        tokens = self.data1[idx]
        x = torch.tensor(tokens[:-1], dtype=torch.long)  # Exclude last token for input
        y = torch.tensor(tokens[1:], dtype=torch.long)   # Exclude first token for output
        
        return x, y

In [5]:
# dataset_path = "/users/eleves-a/2022/brahim.touayouch/.cache/kagglehub/datasets/idevji1/sherlock-holmes-stories/versions/1/sherlock/sherlock"

# file_paths = glob.glob(f"{dataset_path}/*.txt")

# data = [{"text": open(file, "r", encoding="utf-8").read().replace("\n", "").replace("\t", "").replace("    ", "")} for file in file_paths]

# dataset = Dataset.from_list(data)

# tokenizer = BPETokenizer()

# train_dataset = QADataset(dataset, tokenizer=tokenizer, block_size=512)

# x, y = train_dataset[1]
# print(f"Input tokens: {x}")
# print(f"Output tokens: {y}")

In [6]:
# from datasets import load_dataset
# import glob

# dataset_path = "/users/eleves-a/2022/brahim.touayouch/.cache/kagglehub/datasets/idevji1/sherlock-holmes-stories/versions/1/sherlock/sherlock"
# file_paths = glob.glob(f"{dataset_path}/*.txt")

# # Load dataset from the text files directly
# dataset = load_dataset("text", data_files=file_paths)


# tokenizer = BPETokenizer()

# train_dataset = QADataset(dataset['train'], tokenizer=tokenizer, block_size=512)

# x, y = train_dataset[100]
# print(f"Input tokens: {x}")
# print(f"Output tokens: {y}")


In [6]:
data = load_dataset('roneneldan/TinyStories')
pd.DataFrame(data["train"])



Generating validation split: 100%|██████████| 21990/21990 [00:00<00:00, 74590.16 examples/s]


,text
0,"One day, a little girl named Lily found a need..."
1,"Once upon a time, there was a little car named..."
2,"One day, a little fish named Fin was swimming ..."
3,"Once upon a time, in a land full of trees, the..."
4,"Once upon a time, there was a little girl name..."
...,...
2119714,"Once upon a time, in a small town, there lived..."
2119715,"Once upon a time, there was a little boy named..."
2119716,"Once upon a time, there was a big tree. Under ..."
2119717,"Once upon a time, there was a little girl name..."


In [7]:
tokenizer = BPETokenizer()

train_dataset = QADataset(data["train"].select(range(67)), tokenizer=tokenizer, block_size=1024)

x, y = train_dataset[1]
print(f"Input tokens: {x}")
print(f"Output tokens: {y}")

Input tokens: tensor([ 7454,  2402,   257,  ..., 50256, 50256, 50256])
Output tokens: tensor([ 2402,   257,   640,  ..., 50256, 50256, 50256])


/tmp/ipykernel_562539/1717309044.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(tokens[:-1], dtype=torch.long)  # Exclude last token for input
/tmp/ipykernel_562539/1717309044.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(tokens[1:], dtype=torch.long)   # Exclude first token for output


In [7]:
# import kagglehub

# path = kagglehub.dataset_download("idevji1/sherlock-holmes-stories")

# print("Path to dataset files:", path)

# 2. Model Definition

In [8]:
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)
model.to(device)

number of parameters: 124.44M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(

In [7]:
def generate(prompt='', num_samples=1, steps=20, do_sample=True):
    tokenizer = BPETokenizer()

    if prompt == '':
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
    else:
        x = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=len(prompt.split()))['input_ids'].to(device)

    x = x.expand(num_samples, -1)

    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    endoftext_token_id = tokenizer.encoder.encoder['<|endoftext|>']  # Get the ID of <|endoftext|> token

    for i in range(num_samples):
        # Decode the generated tokens
        out = tokenizer.decode(y[i].cpu().squeeze())

        # Check if the output contains the <|endoftext|> token and stop generating when encountered
        out_tokens = y[i].cpu().squeeze().tolist()
        
        # Find the position of <|endoftext|> token in the generated output
        if endoftext_token_id in out_tokens:
            end_pos = out_tokens.index(endoftext_token_id)
            out = tokenizer.decode(y[i].cpu().squeeze()[:end_pos])  # Slice the output up to <|endoftext|> token

        print('\n' + '-' * 80)
        print(out)

In [7]:
# generate(prompt='It all started with the murder of the chief. This is the type of case that Sherlock likes and gets off on.', num_samples=2, steps=50)

# 3. Finetuning With SFT

In [9]:
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4
train_config.batch_size = 1
train_config.max_iters = 20
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [10]:
losses = []
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
        losses.append(trainer.loss.item())
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

Testing DataLoader...
Error while testing DataLoader: list indices must be integers or slices, not list
Training started...


Training Progress:   0%|          | 0/20 [00:00<?, ?it/s]

TypeError: list indices must be integers or slices, not list

: 

In [ ]:
print(losses)

In [ ]:
# model.eval()

In [ ]:
# generate(prompt='This is a test', num_samples=2, steps=50)

In [ ]:
# torch.save(model.state_dict(), 'models/SFT_finetuned.pth')

# 4. Evaluations

In [ ]:
# model.load_state_dict(torch.load('models/SFT_finetuned.pth'))
model.eval()

In [ ]:
tokenizer = BPETokenizer()

def generate_evaluate(prompt='', num_samples=1, steps=20, do_sample=True):
    if prompt == '':
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long).to(device)
    else:
        x = tokenizer(prompt).to(device)

    x = x.expand(num_samples, -1)

    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    generated_responses = []
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        generated_responses.append(out)
    
    return generated_responses

references = []
predictions = []

num_samples = 5

for i, sample in enumerate(data["train"].select(range(num_samples))):
    prompt = sample["instruction"]
    reference = [sample["demonstration"]]
    generated_text = generate_evaluate(prompt=prompt, num_samples=1, steps=50, do_sample=True)[0]

    references.append(reference)
    predictions.append(generated_text)

### 4.1 BLEU Score

In [ ]:
bleu = load("bleu")

bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU Score:", bleu_score)

### 4.2 Rouge Score

In [ ]:
rouge = load("rouge")

scores = rouge.compute(predictions=predictions, references=references)
print("ROUGE Scores:", scores)


In [ ]:
generate_evaluate("what are some good novels for a 9 year old?")

In [ ]:
# load the fine-tuned model

path = "/Data/zakaria.abboud/SFT_finetuned.pth"
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)

model.load_state_dict(torch.load(path))

model.to(device)

model.eval()